# Duckietown Intro to Neural Nets with PyTorch



## Prerequisites

**Step 1:** install pytorch.


**Important:** the following cell is only needed for this notebook. If you're installing pytorch on your own computer (which you totally can, even without a NVIDIA GPU), then please use the instructions from here: https://pytorch.org/get-started/locally/

Note that this will take a few seconds.

In [ ]:
# http://pytorch.org/
!pip install torch==1.13.0 torchvision==0.14.0

**Step 2:** import and check that torch works

In [ ]:
import torch

torch.rand(2)

**Step 3:** Convert from numpy to torch and back

In [ ]:
import numpy as np

a_npy = np.eye(4)
print (a_npy)

a_tor = torch.from_numpy(a_npy) # convert numpy to torch Tensor
print (a_tor)

b_npy = a_tor.numpy() # convert Tensor back to numpy
print (b_npy)

**Step 4:** Notice that if you change the data EITHER in numpy or torch, you're changing it in both

In [ ]:
a_npy[0,3] = 99 # we're only changing the numpy array

a_tor[1,3] = 123 # we're only changing the torch Tensor

print (a_npy) # no we're not
print (a_tor)
print (b_npy)

**Step 5:** Other than that, PyTorch is just like Numpy... but all the things are called different names

In [ ]:
# These things do the same thing, but have different names and one is a property,
# the other one is a function.

print (a_npy.shape)
print (a_tor.size())

# Most (not all) things that you can do in Numpy, you can also do in PyTorch,
# but you have to google what they are called


## Neural Nets

**Step 6:** This is how you define a neural net in PyTorch

**IMPORTANT:** read the comments. It's crucial that you understand what's happening. If something is unclear, ask on stack overflow with the label #neural-nets. It is easy to make simple mistakes when working with neural networks, so it is better to ask for help as early as possible.

In [ ]:
import torch # we already did this, but just in case you want to
# copy-paste this code block

import torch.nn as nn # functions for neural nets, like layers and loss function
import torch.nn.functional as F #  these are helper functions liks sigmoid, relu, etc

# what's missing here is the import for the optimizer


# you need to create a class that inherits from nn.Module

class Net(nn.Module):
    def __init__(self):
        # Here in the init function you only create the layers that you want to
        # use later on. You're not actually connecting them to anything here.
        # So you can create them in any order.

        super(Net, self).__init__()

        # 3 color channels input, 6 convolutional kernels -> 6 channels output,
        # and also 5x5 square kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        # after applying this to a 3x32x32 image without padding or stride,
        # the result will be 6x28x28

        # max pooling with a 2x2 moving window
        self.pool = nn.MaxPool2d(2, 2)
        # after applying this to the 6x28x28 image, the result will be
        # 6x14x14 (the channels are not affected)

        # 6 channels input, 16 convolutional kernels -> 16 channels output,
        # and also 5x5 square kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # after applying this to a 6x14x14 image without padding or stride,
        # the result will be 16x10x10

        # Later in the actual forward pass, we will apply the maxpooling twice,
        # but we only have to define it once, because it doesn't have any para-
        # meters that we're backpropping through.
        # So we know that we will apply MaxPool2d(2,2) again to the 16x10x10 image.
        # Therefore the output of the convolutional layers will be 16x5x5.


        # This layer definition requires that you did the convolution math.
        # The final "image" will be 5 by 5 pixels and 16 channels deep, therefore
        # the input is 16 * 5 * 5.
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # The sizes of these layers are _completely_ arbitrary.
        self.fc2 = nn.Linear(120, 84)

        # ultimately our ouput will be a 10-element vector for each input image
        # which corresponds to a one-hot encoding of a 0-9 integer
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # This function is for a single forward pass through the network.
        # So you get the input, pass it through all the layers you defined above
        # (!important, don't define any new layers here) and then return the final
        # result.

        # Apply, in that order: convolutional layer 1 (3,6,5), ReLU, MaxPool2d(2,2)
        x = self.pool(F.relu(self.conv1(x)))

        # Apply, in that order: convolutional layer 2 (6,16,5), ReLU, MaxPool2d(2,2)
        x = self.pool(F.relu(self.conv2(x)))

        # The input is still 3-dimensional (shape: 16x5x5). Here we transform it
        # into a vector of size (16*5*5 = 400)
        x = x.view(-1, 16 * 5 * 5)

        # Pass it through fully connected layer 1 and then through ReLU
        x = F.relu(self.fc1(x))

        # Pass it through fully connected layer 2 and then through ReLU
        x = F.relu(self.fc2(x))

        # Pass it through the last layer WITHOUT RELU and return it
        x = self.fc3(x)
        return x


# here we just instantiate the network, so we can go use it.
net = Net()

# and make sure it's using 32-bit floats ("Float"), not 64-bit floats ("Double")
net = net.float()


**Step 7 (optional):** print the network

In [ ]:
print (net)

**Step 8:** Test the network with random input

In [ ]:
# create a single image (note how the color channel is in the front)
dummy_input = np.random.uniform(low=0, high=1, size=(3,32,32)).astype(np.float32)

# convert to Tensor:
dummy_tensor = torch.from_numpy(dummy_input)

print ("old size:", dummy_tensor.size())

# IMPORTANT. torch works in batches. You can have a batch size of 1 (if it's only)
# a single image, but you need to add the dimension (which you want to become the
# new axis 0):

dummy_tensor = dummy_tensor.unsqueeze(0)

print ("new size:", dummy_tensor.size(),"<--- see? There's a new first dimension!")

# now we can feed it into the network:
prediction = net(dummy_tensor)

print ("prediction size:",prediction.size(),"<-- The output has the " \
       "same first dimension. That's the batch size!")

print ("")
print (prediction)




**Step 9:** Start optimizing!

In [ ]:
import torch.optim as optim

# instead of Mean Squared Error (MSE or "L2") loss we use CE loss here because
# this has great performance if your output is categorical and ideally in range [0,1]
criterion = nn.CrossEntropyLoss()

# stochastic gradient descent... (There are better options)
# and feed the net.parameters() to the optimizer - that's all the optimizable
# parameters in the netwoprk
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

**Step 10:** Do a trial run of the optimization

In [ ]:
# let's stack ("concatenate" - "torch.cat()") 4 images into a minibatch:
inputs = torch.cat([dummy_tensor, dummy_tensor, dummy_tensor, dummy_tensor], dim=0).float()
print ("inputs.size()", inputs.size())

# let's make some dummy labels - notice how it's only the batch dimension
labels = torch.zeros(4).long()
print ("labels.size()",labels.size())

# zero the parameter gradients (always do this during learning before every forward pass)
optimizer.zero_grad()

# predict the outputs
outputs = net(inputs)
print ("outputs.size()", outputs.size())

# now we have a problem: our labels are unidimensional, but the prediction is
# 10-dimensional (on purpose)... what do?
# Answer: you can either spread out the ground truth into one-hot encoding
# Or: you can use a loss function that can accept both: CrossEntropy.

loss = criterion(outputs, labels) # apply the loss function, notice the format of
# loss(prediction, ground_truth) <-- that's important

# calculate the backpropagation values
loss.backward()

# apply the backprop values according to the optimizer
optimizer.step()

# And print loss - very important - PLOT THIS! If this doesn't go a lot lower
# then you are done and the network is converged
print ("Loss:", loss.item())

# Now run this cell couple of times and watch the loss go down.

## Let's train on real data

**Step 11:** Load the CIFAR-10 dataset

(It's very small and you can download it directly through torch, without manually downloading it)

In [ ]:
# torchvision has some helper functions for image-based dataset
import torchvision

# we want to apply certain things to all of our images - that's what transforms do
import torchvision.transforms as transforms


# we would like all of our incoming images to: become a torch Tensor
# (instead of a numpy array) and we want to normalize all images
# normalization: img = (img-mean)/standard_deviation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # parameters for
# the normalization are mean and std for each channel


# download the training part of the CIFAR-10 dataset and apply transforms
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# wrap that into a multi-threaded data loader for quicker multi-CPU data loading
# and for being able to shuffle the data and sample whole batches and not just
# single elements
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

# same thing for the test dataset

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

# these are the names of the labels, corresponding to 0,1,2,3, etc.
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

**Step 12:** ALWAYS verify your data!

Before running _any_ experiment, always look at your data and make sure it's what you expect it to be

In [ ]:
print ("dataset length:",len(trainset))

image, label = trainset[0]

print ("single image (size):",image.size()) # <-- this is a tensor
print ("single label:",label) # <-- this is not, but it will be if we use the "trainloader" from above


### now let's actually -look- at the images

import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images using the trainloader
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images - make a grid of 4
imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))



In [ ]:
print (images.min(), images.mean(), images.max()) # important to verify your distribution

## Let's glue it all together

**Step 13:** Let's actually run the optimization in a loop with the dataset

In [ ]:
print ("[epoch, line of data]")

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[{}, {}] \tloss: {}'.format(
                epoch + 1,
                i + 1,
                running_loss / 2000)
                )

            running_loss = 0.0

print('Finished Training')


... Now go to the cells bellow (Step 14), and inspect the results. They're not very good, right?

Then, copy the entire previous cell below and run again (to see how the loss converges to something if you run it long enough). Run Step 14 again. See how much better that was? Not perfect, but better.

(Obviously, in a real setting, you'd use some kind of loop instead of copying the cell.)

In [ ]:
# insert code here

**Step 14:** Inspection - manually check your predictions

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# now let's get the net's predictions
outputs = net(images)

# argmax the 10 dimensions into one
_, predicted = torch.max(outputs, 1)

# get the names of the labels for each int label
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

**Step 15:** Actually get a numerical evaluation

In [ ]:
correct = 0
total = 0

# If we're not learning anything then we use the torch.no_grad() environment.
# In this environment no gradients are ever calculated.
with torch.no_grad():
    for data in testloader:

        images, labels = data
        outputs = net(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on the " \
      "10000 test images: {}%".format(100 * correct / total))

~60% is not very accurate. You can do much better.

## "Homework"

**Goal: improve the score, i.e. the test set accuracy**

We won't verify how you do on this, so this is just so you can learn about neural networks.

Ideas:
- try out a different optimizer
- try out more training epochs
- try out a bigger neural network (more hidden nodes or more layers)
- try plotting the loss over time and stop training when the loss converges
- (if you're fancy) try out dataset augmentation - that means applying more transformations to your images before feeding them to the network - like random rotation, random noise, etc. Here's a ist of all transforms: https://pytorch.org/docs/stable/torchvision/transforms.html

# Contributors
 - [Florian Golemo](https://fgolemo.github.io/)
 - [Bhairav Mehta](https://bhairavmehta95.github.io/)
 - [Charlie Gauthier](https://ca.linkedin.com/in/charlie-gauthier)
